In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable as V
from torchvision.models import vgg19
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
import numpy as np
from PIL import Image

ModuleNotFoundError: No module named 'torch'

In [3]:
torch.cuda.set_device(0)

In [8]:
class vgg19_model(nn.Module):
    def __init__(self):
        super(vgg19_model, self).__init__()
        self.model = vgg19(pretrained=True)
        # self.layers = self.forward(model)
        
#         self.batchSize = 1
#         self.num_channel = 3
#         self.img_size = 224
#         self.input = torch.Tensor(self.batchSize, self.num_channel, self.img_size, self.img_size)
        # print(self.layers)

    def forward(self, img):
        pyramid_layers = []
        def extract_feature(self, input, output):
            pyramid_layers.append(output)
            
        relu_idx = [1, 6, 11, 20, 29]
        for i in relu_idx:
            self.model.features[i].register_forward_hook(extract_feature)
        
        # Image preprocessing
        # VGGNet was trained on ImageNet where images are normalized by mean=[0.485, 0.456, 0.406] 
        # and std=[0.229, 0.224, 0.225].
        # We use the same normalization statistics here.
        transform = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                                 std=(0.229, 0.224, 0.225))])
        
        img_preproc = transform(img)
        img_preproc = torch.unsqueeze(img_preproc, 0)
        self.model(V(img_preproc))
        
        return pyramid_layers

In [2]:
vgg = vgg19_model()
im_a = Image.open("../input/dog1.jpg")
im_a_feature = vgg.forward(im_a)
# im_a_feature

NameError: name 'vgg19_model' is not defined

In [ ]:
class Neuron:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"
    def __lt__(self, other):
        return self.x < other.x
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    def __gt__(self, other):
        return self.x > other.x

In [3]:
def RefineSearchRegions(prev_layer_nbbs, receptive_field_radius, feat_width, feat_height):
        """
        Return refined search regions for every p and q in the previous' layer nbbs

        Args:
            prev_layer_nbbs: Previous' layer (l-1) neural best buddies, represented
                as neurons using the Neuron class
            receptive_field_radius: radius of new search regions
                equal to 4 for l = 2,3 and equal to 6 for l = 4, 5
            feat_width: width of feature map for current layer
            feat_height: height of feature map for current layer
        Returns:
            Ps: List containing new P's 
                P = ((r1, c1), (r2, c2))
                where (r1, c1) represent the top left of the search region
                and (r2, c2) represent the bottom right of the search region
            Qs: List containing new Q's
                Q = ((r1, c1), (r2, c2))
                where (r1, c1) represent the top left of the search region
                and (r2, c2) represent the bottom right of the search region
        """
        
        Ps = []
        Qs = []
        
        for p, q in prev_layer_nbbs:
           
            # Top left of search window for P
            P_r1 = max(2 * p.r - receptive_field_radius / 2, 0)
            P_c2 = max(2 * p.c - receptive_field_radius / 2, 0)
            P_bottom_left = Neuron(P_r1, P_c1)
                      
            # Bottom right of search window for P
            P_r2 = min(2 * p.r + receptive_field_radius / 2, feat_width)
            P_c2 = min(2 * p.c + receptive_field_radius / 2, feat_height)
            P_top_right = Neuron(P_r2, P_c2)
            
            # Top left of search window for Q
            Q_r1 = max(2 * q.r - receptive_field_radius / 2, 0)
            Q_c1 = max(2 * q.c - receptive_field_radius / 2, 0)
            Q_bottom_left = Neuron(Q_c1, Q_r1)
            
            # Bottom right of search window for Q                     
            Q_r2 = min(2 * q.r + receptive_field_radius / 2, feat_width)
            Q_c2 = min(2 * q.c + receptive_field_radius / 2, feat_height)
            Q_top_right = Neuron(Q_c2, Q_r2)
            
            # Append P and Q to lists
            Ps.append((P_bottom_left, P_top_right))
            Qs.append((Q_bottom_left, Q_top_right))
        
        return (Ps, Qs)